# Morris Sensitivity Analysis (Elementary Effects Method)

In global sensitivity analysis, the **Morris Method** (also called the *Method of Elementary Effects*) is a screening technique used to identify which input variables in a model have the greatest influence on the output. It was introduced by Max D. Morris in 1991 as a computationally efficient way to explore sensitivity in models with many inputs, without requiring an enormous number of model runs.

The basic idea is:
- Change one input at a time by a small step,
- See how much the output changes,
- Repeat this from different starting points across the input space,
- Summarize how consistently (or inconsistently) each input causes change.

That small one-at-a-time change in an input and the resulting change in the model output is called an **elementary effect**.

---

## Why was Morris developed?

Before Morris (1991), sensitivity analysis often lived in two extremes:

- **Local / derivative-based sensitivity**:  

  This asks “If I make a tiny change to one parameter around the current baseline, how much does the output change?”  
  This is basically a partial derivative.  
  Problem: it only tells you about behavior *near one point* in parameter space, and it assumes smooth / linear behavior.

- **Full global variance-based methods (like Sobol indices)**:  

  These methods try to quantify how much of the total output variance is explained by each input and by their interactions.  
  They’re extremely informative — but also computationally expensive, because they require a lot of model evaluations.

Max Morris was looking for something in-between:

- A method that is **global** (explores the whole parameter space, not just one point),
- But still **cheap enough** to run early, even for high-dimensional problems (many inputs),
- And able to flag inputs that are likely important, nonlinear, or interacting.

So the Morris method is often described as a **screening method**: it’s a first pass that tells you which inputs matter and how they matter, so you know where to focus more detailed analysis later.

---

## What problems does the Morris method solve?

Imagine you have a model:

$$
y = f(x_1, x_2, x_3, \dots, x_k)
$$

where each $x_i$ is an input (a criterion weight, a threshold, a soil parameter, etc.), and $y$ is some decision score (e.g. total suitability, predicted recharge, contaminant load, habitat score).

You want to know:

1. Which inputs have basically **no effect** on $y$? (Those might be safely fixed or ignored.)
2. Which inputs have a **large overall effect** on $y$? (These are important drivers.)
3. Which inputs behave **nonlinearly** or **interact** with other inputs?  
   (For example, “slope only matters once soil permeability is high,” or “forest cover and precipitation together change infiltration potential in a way you don’t get by looking at either alone.”)

The Morris method gives you exactly that information with two summary statistics per input.

---

## Core concept: the Elementary Effect

For each input $x_i$, we define an *elementary effect* as:

$$
EE_i = \frac{f(x_1, \dots, x_i + \Delta, \dots, x_k) - f(x_1, \dots, x_i, \dots, x_k)}{\Delta}
$$

Where:

- $\Delta$ is a small step in the value of $x_i$,
- All other inputs are held constant for that step,
- The numerator is “how much the output changed when we nudged just $x_i$.”

Interpretation:

- $EE_i$ is basically: “If I change only $x_i$ a little, how much does the model output respond?”

But — and this is the key difference from local sensitivity — we don’t do this just once from one baseline. We repeat this from *multiple random locations* in the input space. Each repetition gives us another possible elementary effect for that same input.

So for each input $x_i$, we don’t just get one number. We get a distribution of elementary effects across the space of plausible inputs.

---

## Morris summary metrics

After computing many elementary effects for each input, we summarize them. The two most common summaries are:

1. **$\mu^*$** (mu star):  

   The mean of the *absolute value* of the elementary effects for that input.  
   - High $\mu^*$ means: changing this input tends to cause a big change in the output overall.  
   - This is interpreted as “overall importance” or “influence strength.”

   We use the absolute value so positive and negative effects don’t cancel each other out.

2. **$\sigma$** (sigma):  

   The standard deviation of the elementary effects for that input.  
   - High $\sigma$ means: the effect of this input is not consistent — sometimes it has a big effect, sometimes small, sometimes positive, sometimes negative.  
   - That usually indicates **nonlinearity** or **interactions with other inputs**.

   Intuition: if an input only mattered under certain combinations of other inputs, you’d see a wide spread in its elementary effects → high σ.

This gives you a beautiful diagnostic plot: μ\* on the x-axis (importance) vs σ on the y-axis (interaction / nonlinearity).

- Inputs with **low μ\*** and **low σ** → mostly irrelevant.
- Inputs with **high μ\*** and **low σ** → consistently important, mostly linear effect on the output.
- Inputs with **high μ\*** and **high σ** → important but tricky: nonlinear or involved in interactions.

That’s the classic “Morris scatter plot.”

---

## How it works (conceptually)

1. Define ranges (or distributions) for each input $x_i$.  
   Example: slope weight in WLC could vary from 0.1 to 0.4, precipitation weight from 0.2 to 0.6, etc.

2. Sample a sequence of points in that input space (called “trajectories” or “paths”).  
   Each path walks through the space one input at a time, changing one variable by $\Delta$ while keeping the others fixed, then moving on to the next variable, etc.

3. For each step along that path, compute the elementary effect $EE_i$.

4. Aggregate all the elementary effects for each input across all paths → get $μ^*$ and $\sigma$.

5. Rank or plot inputs based on $\mu^*$ and σ to decide which inputs matter.

This is global because you’re sampling across the full allowable range of inputs — not just perturbing around a single baseline point.

---

## Why this is used

The Morris method is widely used in:

- Environmental modeling and hydrology (e.g., identifying which hydrogeologic parameters most influence recharge estimates or contaminant transport),
- Ecological and habitat suitability modeling,
- Groundwater recharge / infiltration models,
- Flood and erosion models,
- Multi-criteria decision analysis (MCDA), including GIS-based suitability mapping, to see which criteria weights dominate the final suitability score, and where there are strong interactions.

In practice:

- You run Morris first to screen out unimportant variables (so you don’t waste computation on them),
- Then you apply heavier methods (like Sobol variance decomposition) on the variables that survived screening.

So Morris is both:

1. A science tool (which parameters actually matter?),
2. A workflow tool (where should I spend my expensive computation time?).

---

## Summary

- The Morris method is a global, one-factor-at-a-time sensitivity screening method introduced by Max D. Morris in 1991.
- It’s built around **elementary effects**: the change in model output when you nudge one input while holding others constant.
- By repeating that across many starting points, you get a *distribution* of effects for each input.
- You then summarize each input with:
  - **$\mu^*$** (mean absolute elementary effect): how influential this input is overall,
  - **$\sigma$** (stdev of elementary effects): how nonlinear or interaction-heavy its influence is.
- This is incredibly helpful in decision-support models (like WLC suitability mapping) because it tells you:
  - which criteria weights dominate suitability,
  - which ones only matter in combination,
  - and which ones are basically irrelevant.

---

Next, we’ll:

1. Build a tiny synthetic model in Python so you can *see* elementary effects for a toy function,
2. Compute $\mu^*$ and $\sigma$ for each input manually,
3. Reproduce what SALib’s `morris` routines do,
4. Visualize $\mu^*$ vs $\sigma$,
5. Then connect that to a spatial WLC / MCDA setting.

> Now let’s start generating elementary effects for a simple model in Python.